In [1]:
from hashlib import sha256

In [2]:
encrypted_elements = [b'217',b'354',b'466']

## Simple Bloom Encoding Example

In [22]:
num_bits = 32
num_hashfunctions = 4
filterlist = ['0'] * num_bits

for element in encrypted_elements:
    element1 = str.encode('1') + element
    k = sha256(element1).hexdigest()
    h1 = int(k,16) % num_bits

    element2 = str.encode('2') + element
    k = sha256(element2).hexdigest()
    h2 = int(k,16) % num_bits
    
    for i in range(num_hashfunctions):
        pos = ((h1 + i * h2) % num_bits)
        filterlist[num_bits-1-pos]='1'
        
filterstring = ''.join(filterlist)

In [23]:
filterstring

'00000101010011000100001000111100'

In [25]:
for number in range(50):
    check = True

    element1 = str.encode('1') + str.encode('{0:d}'.format(number))
    k = sha256(element1).hexdigest()
    h1 = int(k,16) % num_bits

    element2 = str.encode('2') + str.encode('{0:d}'.format(number))
    k = sha256(element2).hexdigest()
    h2 = int(k,16) % num_bits
    
    for i in range(num_hashfunctions):
        pos = ((h1 + i * h2) % num_bits)
        if (filterlist[num_bits-1-pos]=='1'):
            check = check & True
        else:
            check = False
    if check==True:
        print(number)

14
37
39


## Simple GCS Encoding Example

In [26]:

gcshash_range = 128
gcsdiv = 5

In [27]:
# For all encrypted elements, calculate hash and then bucket value
ulist = []
for element in encrypted_elements:
    k = sha256(element).hexdigest()
    ks = int(k,16) % gcshash_range
    print(ks, k)
    ulist.append(ks)

# Sort the hash bucket values
ulist.sort()
# Calculate deltas between sorted hash bucket values 
udiff = [ulist[0]] + [ulist[n]-ulist[n-1] for n in range(1,len(ulist))]

120 16badfc6202cb3f8889e0f2779b19218af4cbb736e56acadce8148aba9a7a9f8
57 09a1b036b82baba3177d83c27c1f7d0beacaac6de1c5fdcc9680c49f638c5fb9
102 826e27285307a923759de350de081d6218a04f4cff82b20c5ddaa8c60138c066


In [28]:
ulist

[57, 102, 120]

In [29]:
udiff

[57, 45, 18]

In [30]:
# For all delta hash bucket values encode as unary portion for quotient followed by binary for remainder.
# Pad with leading zeros so binary portion is of consistent length.
# Concatenate with previous values

encoded = ''
for diff in udiff:
    if diff != 0:
        quot = int(diff / pow(2,gcsdiv))
        rem = diff % pow(2,gcsdiv)
        next = '{0:b}'.format(rem) + '1' + ('0' * quot)
        pad = next.zfill(quot+gcsdiv+1)
        print(quot,rem, next, pad)
        encoded = pad + encoded

1 25 1100110 1100110
1 13 110110 0110110
0 18 100101 100101


In [31]:
encoded

'10010101101101100110'